<a href="https://colab.research.google.com/github/osullik/bc-autoreporter/blob/main/Summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [157]:
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [158]:
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
from transformers import BartTokenizer, BartForConditionalGeneration
import numpy as np
import torch
import random
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [160]:
device = torch.device("cpu")

if torch.cuda.is_available():
   print("Using GPU")
   device = torch.device("cuda:0")

Using GPU


# Read in data
Assume each set of evaluations corresponds to the same employee. Different employees will be read in separately and run through the model separately to generate per-employee summaries.

In [161]:
original_text = "On January 15th, 2021, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his successful completion of a routine safety check on Reactor 4. His actions show #competence #attentiontodetail #safetyconsciousness. On February 2nd, 2021, @HomerSimpson was observed to perform to a poor standard. This was evidenced by his failure to properly label hazardous waste containers in the storage room, leading to a safety hazard. His actions show #carelessness #lackofattention #safetyoversight. On March 12th, 2021, @HomerSimpson was observed to perform to an excellent standard. This was evidenced by his quick thinking and calm demeanor during an unexpected power surge in the control room. His actions show #resilience #problem-solving #teamwork.On April 5th, 2021, @HomerSimpson was observed to perform to a very good standard. This was evidenced by his leadership during a fire drill, guiding his team to safety and ensuring that all necessary procedures were followed. His actions show #leadership #teamcoordination #safetyconsciousness. On May 22nd, 2021, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his successful completion of a routine maintenance check on the cooling system, though he did require some prompting to remember certain procedures. His actions show #competence #teachable #attentiontodetail. On June 7th, 2021, @HomerSimpson was observed to perform to a poor standard. This was evidenced by his failure to report a leak in a coolant pipe, which caused damage to nearby equipment. His actions show #negligence #lackofresponsibility #safetyoversight. On July 15th, 2021, @HomerSimpson was observed to perform to a good standard. This was evidenced by his successful completion of a new training module on emergency response procedures. His actions show #willingnesstolearn #adaptability #safetyconsciousness. On August 9th, 2021, @HomerSimpson was observed to perform to a very good standard. This was evidenced by his coordination of a team to repair a malfunctioning steam valve, minimizing downtime and avoiding any safety hazards. His actions show #teamcoordination #problem-solving #efficiency. On September 27th, 2021, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his successful completion of a routine inspection of the turbine generators, though he did forget to record one measurement. His actions show #competence #attentiontodetail #teachable. On October 14th, 2021, @HomerSimpson was observed to perform to an excellent standard. This was evidenced by his quick response to a power outage in Sector 7G, restoring power and ensuring that critical systems remained operational. His actions show #resilience #problem-solving #safetyconsciousness. On May 3rd @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his willingness to work overtime to assist with the cleanup of the oil spill. His actions show #dedication and #cooperativeness. On May 11th @HomerSimpson was observed to perform to a poor standard. This was evidenced by his careless handling of nuclear waste, which resulted in a spill that contaminated the town's water supply. His actions show #recklessness and #carelessness. On May 27th @HomerSimpson was observed to perform to a poor standard. This was evidenced by his lack of concern for the safety of his fellow workers when he removed safety equipment in order to increase his productivity. His actions show #irresponsibility and #selfishness. On June 3rd @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his ability to work effectively under pressure when the plant's emergency systems were activated. His actions show #calmness and #levelheadedness. On June 9th @HomerSimpson was observed to perform to a good standard. This was evidenced by his creative solution to the plant's budget problems, which involved turning off unnecessary equipment. His actions show #innovation and #costconsciousness. On June 16th @HomerSimpson was observed to perform to a very good standard. This was evidenced by his success in negotiating a union contract that was fair to both the workers and the company. His actions show #diplomacy and #negotiation skills. On June 23rd @HomerSimpson was observed to perform to a poor standard. This was evidenced by his careless use of the company car, which resulted in damage to both the car and the property of others. His actions show #carelessness and #negligence. On June 29th @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his willingness to work on a holiday in order to assist with an important project. His actions show #dedication and #teamwork. On August 1st, while filling in for Smithers, @HomerSimpson was observed to perform to a very good standard. This was evidenced by his organization of Mr. Burns' schedule and preparation of his meals. His actions show #responsibility, #efficiency, and #attentiontodetail. On August 3rd, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his handling of Mr. Burns' incoming mail and scheduling of appointments. His actions show #competence and #accuracy, but there is room for improvement. On August 7th, @HomerSimpson was observed to perform to a poor standard. This was evidenced by his mishandling of Mr. Burns' laundry, resulting in several ruined garments. His actions show #carelessness and #inattention. On August 10th, @HomerSimpson was observed to perform to a very good standard. This was evidenced by his creative solution to a scheduling conflict, allowing Mr. Burns to attend both a charity event and a business meeting. His actions show #resourcefulness and #innovation. On August 15th, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his coordination of Mr. Burns' transportation for the day. His actions show #adequacy and #dependability, but once again, there is room for improvement. On August 18th, @HomerSimpson was observed to perform to a good standard. This was evidenced by his successful negotiation of a contract with a new supplier, resulting in cost savings for the company. His actions show #assertiveness and #businesssavvy. On August 22nd, @HomerSimpson was observed to perform to a very good standard. This was evidenced by his management of Mr. Burns' finances, including payment of bills and tracking of expenses. His actions show #financialacumen and #thoroughness. On August 25th, @HomerSimpson was observed to perform to a poor standard. This was evidenced by his forgetting to inform Mr. Burns of an important meeting, resulting in a missed opportunity for the company. His actions show #forgetfulness and #lackofattention. On August 28th, @HomerSimpson was observed to perform to a good standard. This was evidenced by his troubleshooting of a technical issue with the office equipment, avoiding a costly repair bill. His actions show #technicalskill and #problem-solving. On August 31st, @HomerSimpson was observed to perform to a satisfactory standard. This was evidenced by his handling of Mr. Burns' correspondence and phone calls for the day. His actions show #mediocrity and #routine, but no major mistakes were made."

# Preprocessing Functions

In [162]:
# def clean_text(text):
#     text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
#     text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
#     text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
#     text = re.sub('\t', ' ', text)
#     text = re.sub(r" +", ' ', text)
#     return text

In [163]:
# def shuffle_sentences(text):
#   x = text.split('.')
#   random.shuffle(x)
#   shuffled = ' '.join(x)
#   return "summarize: " + shuffled 

In [127]:
# generate chunks of text \ sentences <= 512 tokens
def nest_sentences(document):
  nested = []
  sent = ["summarize"]  # need to provide task instruction as the first token
  length = 0
  for sentence in nltk.sent_tokenize(document):
    length += len(sentence)
    if length < 512:
      sent.append(sentence)
    else:
      nested.append(sent)
      sent = []
      length = 0

  if sent:
    random.shuffle(sent)
    result = ''.join(sent)
    nested.append(result)
  return nested

# Summarization function
Nested function feeds input sentence by sentence, which doesn't allow summarization across a report. Use generate_summary instead.

In [ ]:
# from https://github.com/saarthdeshpande/book-summarizer/blob/master/model.py

# def generate_summary(text, tokenizer, model):
#     preprocess_text = text.strip().replace("\n", "")
#     t5_prepared_Text = "summarize: " + preprocess_text
#     tokenized_text = tokenizer.encode(t5_prepared_Text, truncation=True, return_tensors="pt")
#     print("Num toks = ", len(preprocess_text))
#     tokenized_text = tokenized_text.to(device)

#     summary_ids = model.generate(tokenized_text,
#                                  num_beams=5,
#                                  no_repeat_ngram_size=2,
#                                  min_length=30,
#                                  max_length=96,
#                                  early_stopping=True)

#     output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return output

In [164]:
# generate summary on text with <= 512 tokens
def generate_summary_nested(nested_sentences, tokenizer, model):
  summaries = []
  for nested in nested_sentences:
    input_tokenized = tokenizer.encode(' '.join(nested), truncation=True, return_tensors='pt')
    input_tokenized = input_tokenized.to(device)
    summary_ids = model.to(device).generate(input_tokenized,
                                            num_beams=4,
                                            no_repeat_ngram_size=2,
                                            min_length=5,
                                            max_length=100,
                                            early_stopping=True)
    output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

In [165]:
# generate summary on text with <= 512 tokens
def generate_summary(sentences, tokenizer, model):
  input_tokenized = tokenizer.encode(sentences, truncation=True, return_tensors='pt')
  input_tokenized = input_tokenized.to(device)
  summary_ids = model.to(device).generate(input_tokenized,
                                          num_beams=4,
                                          min_length=50,
                                          max_length=1000,
                                          early_stopping=True)
  output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
  return output

# Create Tokenizer
Corresponding T5 tokenizer from Hugging Face library

In [166]:
# Instantiate the tokenizer 
tokenizer = T5Tokenizer.from_pretrained('t5-small')
##tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
print(tokenizer.model_max_length)

512


/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [167]:
# # Concatenate the word "summarize:" to raw text to tell the model what task to perform on the input
# preprocess_text = original_text.strip().replace("\n","")
# t5_prepared_Text = "summarize: "+preprocess_text
# print ("original text preprocessed: \n", preprocess_text)

# tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt")

# Create Model
Pretrained T5 model from Hugging Face library. This is a generic model that can do translation, summarization, etc. We will pass it the 'summarize' command appended to the front of the input stream to trigger a summary output from the model.

https://towardsdatascience.com/simple-abstractive-text-summarization-with-pretrained-t5-text-to-text-transfer-transformer-10f6d602c426 

In [168]:
# Instantiate the model 
model = T5ForConditionalGeneration.from_pretrained('t5-small')
##model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)

# Summarize
Generate summary text given preprocessed input text containing a set of reports for a given employee.

In [169]:
nested_text = nest_sentences(original_text)
inputs = []
for i in nested_text[0]:
  inputs.append(i.split())  # TODO loop over all the chunks to get a long summar for each employee
inputs_squeezed = np.concatenate( inputs, axis=0 )
inputs_squeezed = [i + ' ' for i in inputs_squeezed]

On January 15th, 2021, @HomerSimpson was observed to perform to a satisfactory standard.
This was evidenced by his successful completion of a routine safety check on Reactor 4.
His actions show #competence #attentiontodetail #safetyconsciousness.
On February 2nd, 2021, @HomerSimpson was observed to perform to a poor standard.
This was evidenced by his failure to properly label hazardous waste containers in the storage room, leading to a safety hazard.
His actions show #carelessness #lackofattention #safetyoversight.
On March 12th, 2021, @HomerSimpson was observed to perform to an excellent standard.
This was evidenced by his quick thinking and calm demeanor during an unexpected power surge in the control room.
His actions show #resilience #problem-solving #teamwork.On April 5th, 2021, @HomerSimpson was observed to perform to a very good standard.
This was evidenced by his leadership during a fire drill, guiding his team to safety and ensuring that all necessary procedures were followed

In [170]:
# Summarize preprocessed text
generate_summary(' '.join(inputs_squeezed), tokenizer, model)

['@HomerSimpson was observed to perform to a satisfactory standard . his actions show #competence #attentiontodetail #safetyconsciousness . this was evidenced by his failure to properly label hazardous waste containers in the storage room .']

In [ ]:
# # summmarize 
# summary_ids = model.generate(tokenized_text,
#                                     num_beams=4,
#                                     no_repeat_ngram_size=2,
#                                     min_length=500,
#                                     max_length=1000,
#                                     early_stopping=True)

# output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# print ("\n\nSummarized text: \n",output)



Summarized text: 
 @HomerSimpson was observed to perform to a satisfactory standard. this was evidenced by his failure to properly label hazardous waste containers in the storage room, leading to safety hazard. his actions show #resilience #problem-solving #safetyoversight. on may 27th, 2021, @homersimpedson performed to an excellent standard. he did forget to record one measurement, resulting in several ruined garments, and his efforts show'responsibility........................................ in dr st. pep'd o e w t y, the! &, in an un ned,,' ; i m com u.s. and re, his action show self-help, on august, at the end? raf was seen if the company's own – - and not to the safety of l', an alleged thoroughness / and #sys and the same : b p f, to his en, her, or the, that ’s to have c., #selfishness and/ or an, but.... to get his work despite wards vs the in his, "the......... and... on g z x and other %, this â€” his own,......and..... and, of, just ten ng, with the work,#self — ­,’ ing 